In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import os

# Evaluation scripts
from CompetitionEvaluation import load_data, structure_data, calculate_metrics

In [28]:
def get_actuals(year):
    df_cm_actuals_year = pd.read_parquet(f"actuals/cm/window=Y{year}/cm_actuals_{year}.parquet")
    df_cm_actuals_year['ged_sb'] = df_cm_actuals_year['ged_sb'].astype('int32')
    df_cm_actuals_year.rename(columns={"ged_sb": "outcome"}, errors="raise", inplace=True)
    df_cm_actuals_year['year'] = year
    # append month. each year has 12 months. iterate according to month_id
    start_month = df_cm_actuals_year['month_id'].min()
    mapping_month = {start_month + month_i: month_i for month_i in range(0, 12)}
    df_cm_actuals_year['month'] = df_cm_actuals_year['month_id'].map(mapping_month)

    return df_cm_actuals_year


df_cm_actuals_2018 = get_actuals(year=2018)
df_cm_actuals_2019 = get_actuals(year=2019)
df_cm_actuals_2020 = get_actuals(year=2020)
df_cm_actuals_2021 = get_actuals(year=2021)
df_cm_actuals_2018

,month_id,country_id,outcome,year,month
0,457,1,0,2018,0
1,457,2,0,2018,0
2,457,3,0,2018,0
3,457,4,0,2018,0
4,457,5,0,2018,0
...,...,...,...,...,...
2287,468,242,0,2018,11
2288,468,243,0,2018,11
2289,468,244,0,2018,11
2290,468,245,9,2018,11


In [29]:
# submission/benchmark_last_historical_poisson
submissions_to_concern = ["benchmark_last_historical_poisson", "benchmark_boostrap",
                          "ng_boost_cm_v1.0_pw_14_normal_d_20_n_300_s_crpscore_c_T_m_T_bsd_5_mbf_0.5"]


def read_metric(submisson, metric):
    metric_all_years = []
    for y in range(2018, 2023):
        try:
            # submission/benchmark_last_historical_poisson/eval/cm/window=Y2018/metric=crps/crps.csv
            df = pd.read_csv(f"submission/{submisson}/eval/cm/window=Y{y}/metric={metric}/{metric}.csv")
        except FileNotFoundError:
            print(f"File not found for {submisson} {metric} {y}")
            continue
        df['year'] = y
        start_month = df['month_id'].min()
        mapping_month = {start_month + month_i: month_i for month_i in range(0, 12)}
        df['month'] = df['month_id'].map(mapping_month) + 1
        metric_all_years.append(df)
    # concat all years
    df_metric_all_years = pd.concat(metric_all_years)
    df_metric_all_years.reset_index()
    return df_metric_all_years


def metric_per_month(df_metric, metric, include_2022=False):
    if not include_2022:
        df_metric = df_metric[df_metric['year'] != 2022]
    res = df_metric.groupby(['month'])['value'].mean().reset_index().set_index('month')
    res.rename(columns={"value": metric}, inplace=True)
    return res


def metric_per_month_for_2022(df_metric, metric):
    df_metric = df_metric[df_metric['year'] == 2022]
    if df_metric.empty:
        return "N/A"
    res = df_metric.groupby(['month'])['value'].mean().reset_index().set_index('month')
    res.rename(columns={"value": metric}, inplace=True)
    return res


def metric_per_country(df_metric, metric, include_2022=False):
    res = df_metric.groupby(['country_id'])['value'].mean().reset_index().set_index('country_id')
    res.rename(columns={"value": metric}, inplace=True)
    return res


def metric_mean(df_metric, metric, per_year=False, include_2022=False):
    """return mean all years"""
    if per_year:
        res = df_metric.groupby(['year'])['value'].mean().reset_index().set_index('year')
        res.rename(columns={"value": metric}, inplace=True)
        return res
    else:
        return df_metric['value'].mean()


submissions_metrics = []

for submission_to_concern in submissions_to_concern:
    submission_metadata = {'name': submission_to_concern}

    # get CRPS metrics
    crps_submission = read_metric(submission_to_concern, "crps")
    submission_metadata['crps_mean'] = metric_mean(crps_submission, "crps")
    submission_metadata['crps_mean_year'] = metric_mean(crps_submission, "crps", per_year=True)
    submission_metadata['crps_per_month'] = metric_per_month(crps_submission, "crps", )
    submission_metadata['crps_per_month_2022'] = metric_per_month_for_2022(crps_submission, "crps")
    submission_metadata['crps_per_country'] = metric_per_country(crps_submission, "crps")

    ign_submission = read_metric(submission_to_concern, "ign")
    submission_metadata['ign_mean'] = metric_mean(ign_submission, "ign")
    submission_metadata['ign_mean_year'] = metric_mean(ign_submission, "ign", per_year=True)
    submission_metadata['ign_per_month'] = metric_per_month(ign_submission, "ign")
    submission_metadata['ign_per_month_2022'] = metric_per_month_for_2022(ign_submission, "ign")
    submission_metadata['ign_per_country'] = metric_per_country(ign_submission, "ign")

    mis_submission = read_metric(submission_to_concern, "mis")
    submission_metadata['mis_mean'] = metric_mean(mis_submission, "mis")
    submission_metadata['mis_mean_year'] = metric_mean(mis_submission, "mis", per_year=True)
    submission_metadata['mis_per_month'] = metric_per_month(mis_submission, "mis")
    submission_metadata['mis_per_month_2022'] = metric_per_month_for_2022(mis_submission, "mis")
    submission_metadata['mis_per_country'] = metric_per_country(mis_submission, "mis")

    submissions_metrics.append(submission_metadata)
    print(submission_metadata['crps_mean'], submission_metadata['ign_mean'], submission_metadata['mis_mean'])

submission_metadata['mis_mean_year']

File not found for benchmark_last_historical_poisson crps 2022
File not found for benchmark_last_historical_poisson ign 2022
File not found for benchmark_last_historical_poisson mis 2022
20.127276400523563 1.1349818975297945 383.1922774869111
File not found for benchmark_boostrap crps 2022
File not found for benchmark_boostrap ign 2022
File not found for benchmark_boostrap mis 2022
25.721012127072424 1.104536244633859 492.36275632635255
28.96012747984293 1.01202212324908 456.8056588132637


,mis
year,
2018,181.643412
2019,152.698102
2020,155.546051
2021,461.702640
2022,1332.438089


# Assembling evaluation tables

In [31]:
# Create table A for each model with:
# one row for each year plus one for mean over years
# one col for crps 
# one col for ign 
# one col for mis
# Create table B with the same, but metrics per month

overleafpath = "overleaf_tables/"
if not os.path.exists(overleafpath):
    os.makedirs(overleafpath)

# Table A:
for submission_metrics in submissions_metrics:
    generate_2022_table = False
    if submission_metrics['name'] == submissions_to_concern[2]:
        generate_2022_table = True
    table_per_year = pd.concat([submission_metrics['crps_mean_year'], submission_metrics['ign_mean_year'],
                                submission_metrics['mis_mean_year']], axis=1)
    table_per_year.loc['Mean 2018-2021'] = table_per_year[0:3].mean()

    if generate_2022_table:
        table_per_year.loc['Mean 2018-2022'] = table_per_year.mean()
    else:
        table_per_year.loc['Mean 2018-2022'] = "N/A"

    table_filename = overleafpath + submission_metrics['name'] + '_yearly' + '.tex'
    print(submission_metrics['name'], table_filename)
    with open(table_filename, 'w') as tf:
        tf.write(table_per_year.to_latex(float_format="{:.2f}".format))

# Table B:
for submission_metrics in submissions_metrics:
    table_monthly = pd.concat([submission_metrics['crps_per_month'], submission_metrics['ign_per_month'],
                               submission_metrics['mis_per_month']], axis=1)
    # table_monthly.loc['Mean'] = table_monthly.mean()
    table_filename = overleafpath + submission_metrics['name'] + '_monthly' + '.tex'
    print(submission_metrics['name'], table_filename)
    with open(table_filename, 'w') as tf:
        tf.write(table_monthly.to_latex(float_format="{:.2f}".format))

# Table B for 2022 only
submissions_supports_2022 = [submissions_metrics[2], ]
for submission_metrics in submissions_supports_2022:
    table_monthly = pd.concat([submission_metrics['crps_per_month_2022'],
                               submission_metrics['ign_per_month_2022'],
                               submission_metrics['mis_per_month_2022']], axis=1)
    table_monthly.loc['Mean'] = table_monthly.mean()
    table_filename = overleafpath + submission_metrics['name'] + '_monthly_2022' + '.tex'
    print(submission_metrics['name'], table_filename)
    with open(table_filename, 'w') as tf:
        tf.write(table_monthly.to_latex(float_format="{:.2f}".format))



benchmark_last_historical_poisson overleaf_tables/benchmark_last_historical_poisson_yearly.tex
benchmark_boostrap overleaf_tables/benchmark_boostrap_yearly.tex
ng_boost_cm_v1.0_pw_14_normal_d_20_n_300_s_crpscore_c_T_m_T_bsd_5_mbf_0.5 overleaf_tables/ng_boost_cm_v1.0_pw_14_normal_d_20_n_300_s_crpscore_c_T_m_T_bsd_5_mbf_0.5_yearly.tex
benchmark_last_historical_poisson overleaf_tables/benchmark_last_historical_poisson_monthly.tex
benchmark_boostrap overleaf_tables/benchmark_boostrap_monthly.tex
ng_boost_cm_v1.0_pw_14_normal_d_20_n_300_s_crpscore_c_T_m_T_bsd_5_mbf_0.5 overleaf_tables/ng_boost_cm_v1.0_pw_14_normal_d_20_n_300_s_crpscore_c_T_m_T_bsd_5_mbf_0.5_monthly.tex
ng_boost_cm_v1.0_pw_14_normal_d_20_n_300_s_crpscore_c_T_m_T_bsd_5_mbf_0.5 overleaf_tables/ng_boost_cm_v1.0_pw_14_normal_d_20_n_300_s_crpscore_c_T_m_T_bsd_5_mbf_0.5_monthly_2022.tex


In [ ]:
table_per_year

In [ ]:
table_monthly